In [16]:
import torch
import numpy as np
from sklearn.decomposition import PCA
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from numpy.core.multiarray import _reconstruct
import torch.serialization

# Allowlist the _reconstruct function if you trust the source
torch.serialization.add_safe_globals([_reconstruct])

# Now attempt to load with weights_only=True
train_data, train_labels = torch.load('D:\\ASU\\Fall 2024\\EEE549\\Final term project\\sml-final\\data\\preprocessed\\cifar10\\preprocessed_train_cifar.pt', weights_only=False)
test_data, test_labels = torch.load('D:\\ASU\\Fall 2024\\EEE549\\Final term project\\sml-final\\data\\preprocessed\\cifar10\\preprocessed_test_cifar.pt', weights_only=False)


In [8]:
print(type(train_data))
print(isinstance(train_data, torch.Tensor))

<class 'numpy.ndarray'>
False


In [12]:
# Check the data types
print(f'Train Data Type: {type(train_data)}')
print(f'Test Data Type: {type(test_data)}')

# Assuming the data is loaded correctly and is indeed a Tensor
if isinstance(train_data, torch.Tensor) and isinstance(test_data, torch.Tensor):
    # Flatten the data and convert to numpy arrays for PCA
    X_train = train_data.view(train_data.size(0), -1).numpy()
    X_test = test_data.view(test_data.size(0), -1).numpy()
else:
    print("Data is not in tensor format.")

Train Data Type: <class 'torch.Tensor'>
Test Data Type: <class 'torch.Tensor'>


In [10]:
if not isinstance(train_data, torch.Tensor):
    train_data = torch.tensor(train_data, dtype=torch.float)
if not isinstance(test_data, torch.Tensor):
    test_data = torch.tensor(test_data, dtype=torch.float)

In [11]:
# Flatten the data and convert to numpy arrays for PCA
X_train = train_data.view(train_data.size(0), -1).numpy()
X_test = test_data.view(test_data.size(0), -1).numpy()

# Apply PCA to reduce dimensions
pca = PCA(n_components=150)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Convert PCA data back to tensors for training
X_train_pca = torch.tensor(X_train_pca, dtype=torch.float)
X_test_pca = torch.tensor(X_test_pca, dtype=torch.float)
y_train = torch.tensor(train_labels, dtype=torch.long)
y_test = torch.tensor(test_labels, dtype=torch.long)

C:\Users\Aditya\AppData\Local\Temp\ipykernel_49764\1025937828.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(train_labels, dtype=torch.long)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_49764\1025937828.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(test_labels, dtype=torch.long)


## PCA

In [21]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(150, 64)  # 150 features from PCA
        self.fc2 = nn.Linear(64, 10)   # Output 10 classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create the neural network
model = SimpleNN()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Data conversion and DataLoader setup
X_train_pca_tensor = torch.tensor(X_train_pca, dtype=torch.float)
y_train_tensor = torch.tensor(train_labels, dtype=torch.long)
X_test_pca_tensor = torch.tensor(X_test_pca, dtype=torch.float)
y_test_tensor = torch.tensor(test_labels, dtype=torch.long)

train_dataset = TensorDataset(X_train_pca_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_pca_tensor, y_test_tensor)
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Function to calculate accuracy
def calculate_accuracy(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    train_acc = calculate_accuracy(model, trainloader)
    test_acc = calculate_accuracy(model, testloader)
    train_error = running_loss / len(trainloader)
    test_error = criterion(model(X_test_pca_tensor), y_test_tensor).item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_error:.4f}, Test Loss: {test_error:.4f}, Train Acc: {train_acc:.2f}%, Test Acc: {test_acc:.2f}%')


C:\Users\Aditya\AppData\Local\Temp\ipykernel_49764\14249072.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_pca_tensor = torch.tensor(X_train_pca, dtype=torch.float)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_49764\14249072.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(train_labels, dtype=torch.long)
C:\Users\Aditya\AppData\Local\Temp\ipykernel_49764\14249072.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_pca_tensor = torch.tensor(X_test_pca, dtype=torch.float)
C:\Users\Adit

Epoch 1/100, Train Loss: 12.8325, Test Loss: 4.8035, Train Acc: 74.43%, Test Acc: 70.40%
Epoch 2/100, Train Loss: 3.0200, Test Loss: 3.3831, Train Acc: 76.99%, Test Acc: 71.75%
Epoch 3/100, Train Loss: 1.6870, Test Loss: 2.2692, Train Acc: 82.31%, Test Acc: 74.05%
Epoch 4/100, Train Loss: 1.1009, Test Loss: 2.2265, Train Acc: 79.78%, Test Acc: 72.05%
Epoch 5/100, Train Loss: 0.7321, Test Loss: 1.4460, Train Acc: 85.17%, Test Acc: 74.55%
Epoch 6/100, Train Loss: 0.5595, Test Loss: 1.1480, Train Acc: 88.04%, Test Acc: 75.80%
Epoch 7/100, Train Loss: 0.4191, Test Loss: 1.0887, Train Acc: 87.85%, Test Acc: 74.25%
Epoch 8/100, Train Loss: 0.3880, Test Loss: 1.0956, Train Acc: 87.27%, Test Acc: 73.60%
Epoch 9/100, Train Loss: 0.3522, Test Loss: 0.9874, Train Acc: 90.31%, Test Acc: 76.00%
Epoch 10/100, Train Loss: 0.3133, Test Loss: 0.9029, Train Acc: 91.24%, Test Acc: 77.45%
Epoch 11/100, Train Loss: 0.2966, Test Loss: 0.9741, Train Acc: 90.62%, Test Acc: 74.65%
Epoch 12/100, Train Loss: 0.2

## CNN

In [17]:
train_data = train_data.reshape((-1, 3, 32, 32))
test_data = test_data.reshape((-1, 3, 32, 32))

# Create TensorDatasets
train_dataset = TensorDataset(torch.tensor(train_data, dtype=torch.float), torch.tensor(train_labels, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(test_data, dtype=torch.float), torch.tensor(test_labels, dtype=torch.long))

# Create DataLoaders
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

C:\Users\Aditya\AppData\Local\Temp\ipykernel_49764\3918409937.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(train_data, dtype=torch.float), torch.tensor(train_labels, dtype=torch.long))
C:\Users\Aditya\AppData\Local\Temp\ipykernel_49764\3918409937.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_dataset = TensorDataset(torch.tensor(test_data, dtype=torch.float), torch.tensor(test_labels, dtype=torch.long))


In [23]:
# Assuming the original CIFAR-10 data has been loaded into train_data and test_data with proper image shapes
train_dataset = TensorDataset(torch.tensor(train_data.reshape(-1, 3, 32, 32), dtype=torch.float), torch.tensor(train_labels, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(test_data.reshape(-1, 3, 32, 32), dtype=torch.float), torch.tensor(test_labels, dtype=torch.long))

trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize CNN, loss function, and optimizer
cnn = CNN()
criterion_cnn = nn.CrossEntropyLoss()
optimizer_cnn = optim.Adam(cnn.parameters(), lr=0.001)

# Function to calculate accuracy
def calculate_accuracy(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for data in data_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    model.train()  # Switch back to training mode
    return 100 * correct / total

# Assuming trainloader and testloader are defined
# Training loop for CNN
for epoch in range(10):
    cnn.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        optimizer_cnn.zero_grad()
        outputs = cnn(inputs)
        loss = criterion_cnn(outputs, labels)
        loss.backward()
        optimizer_cnn.step()
        running_loss += loss.item()

    train_acc = calculate_accuracy(cnn, trainloader)
    test_acc = calculate_accuracy(cnn, testloader)
    train_loss = running_loss / len(trainloader)
    test_loss = 0.0
    for data in testloader:
        images, labels = data
        outputs = cnn(images)
        loss = criterion_cnn(outputs, labels)
        test_loss += loss.item()
    test_loss /= len(testloader)

    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Train Acc: {train_acc:.2f}%, Test Acc: {test_acc:.2f}%')


C:\Users\Aditya\AppData\Local\Temp\ipykernel_49764\926196593.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(train_data.reshape(-1, 3, 32, 32), dtype=torch.float), torch.tensor(train_labels, dtype=torch.long))
C:\Users\Aditya\AppData\Local\Temp\ipykernel_49764\926196593.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_dataset = TensorDataset(torch.tensor(test_data.reshape(-1, 3, 32, 32), dtype=torch.float), torch.tensor(test_labels, dtype=torch.long))


Epoch 1, Train Loss: 0.9729, Test Loss: 0.5906, Train Acc: 68.31%, Test Acc: 68.75%
Epoch 2, Train Loss: 0.5476, Test Loss: 0.5035, Train Acc: 76.83%, Test Acc: 75.70%
Epoch 3, Train Loss: 0.5015, Test Loss: 0.5485, Train Acc: 75.69%, Test Acc: 74.40%
Epoch 4, Train Loss: 0.4713, Test Loss: 0.5479, Train Acc: 76.94%, Test Acc: 73.55%
Epoch 5, Train Loss: 0.4511, Test Loss: 0.5057, Train Acc: 80.07%, Test Acc: 75.80%
Epoch 6, Train Loss: 0.4218, Test Loss: 0.5048, Train Acc: 81.66%, Test Acc: 76.65%
Epoch 7, Train Loss: 0.4001, Test Loss: 0.4922, Train Acc: 82.09%, Test Acc: 76.35%
Epoch 8, Train Loss: 0.3870, Test Loss: 0.4933, Train Acc: 85.83%, Test Acc: 77.25%
Epoch 9, Train Loss: 0.3720, Test Loss: 0.4834, Train Acc: 86.97%, Test Acc: 77.95%
Epoch 10, Train Loss: 0.3526, Test Loss: 0.5203, Train Acc: 85.10%, Test Acc: 76.05%
